# Author : Anand Ajmera, Jose Carlos Mayoral Banos


In [96]:
import numpy as np
import cv2
import scipy.spatial
import os,sys
import PIL.Image
import scipy.spatial.distance
import matplotlib.pyplot as plt

In [39]:
data = np.genfromtxt('/home/anand/Studies/SEM2/Robot_Perception/Week9/PGPShelf/PGPShelf/shelf0straightwithLinesPoints.csv',delimiter=',')

data.shape

(14, 3)

In [33]:
jpgfile = PIL.Image.open("/home/anand/Studies/SEM2/Robot_Perception/Week9/PGPShelf/PGPShelf/shelf0straightwithLines.jpg")
img_data = np.array(PIL.Image.open("/home/anand/Studies/SEM2/Robot_Perception/Week9/PGPShelf/PGPShelf/shelf0straightwithLines.jpg"))


In [43]:
new_data = data[:,1:3]
new_data


array([[  156.,   156.],
       [ 3067.,    97.],
       [ 3074.,   115.],
       [ 3044.,  1491.],
       [ 2285.,  1815.],
       [  899.,  1798.],
       [ 2289.,  1420.],
       [  890.,  1416.],
       [  155.,  1119.],
       [  188.,  2418.],
       [  220.,  1784.],
       [  180.,   323.],
       [  824.,  1048.],
       [ 2369.,  1048.]])

In [84]:
# Given points
b1 = new_data[10,:]
t1 = new_data[11,:]
b2 = new_data[4,:]
t2 = new_data[6,:]
h_p1 = new_data[12,:]
h_p2 = new_data[13,:]


In [3]:
def make_homog(point):
    if(len(point) == 2):
        return np.append(point,[1])
    elif(len(point) == 3):
        point = np.array(point,dtype=float)
        point = point/point[-1]
        return point
    else:
        print "Unknown size of point " , point
        return -1



In [85]:
m = make_homog

b1_ = m(b1)
t1_ = m(t1)

b2_ = m(b2)
t2_ = m(t2)

h_p1_ = m(h_p1)
h_p2_ = m(h_p2)

def distance_from_b2(point,second_point = b2_):
    return np.array([scipy.spatial.distance.euclidean(point[0:2],second_point[0:2]),1])

In [105]:
#Find line l through h_p1 and h_p2
l = np.cross(h_p2_,h_p1_)
l_ = make_homog(l)
print "l : " , l_


#Find line through b1 and b2
l_b1_b2 = np.cross(b1_,b2_)
l_b1_b2 = make_homog(l_b1_b2)
print "l_b1_b2 : " , l_b1_b2

# Find u by the intersection of l_b1_b2 and l_
u_inf = np.cross(l_b1_b2,l_)
u_inf_ = make_homog(u_inf)
print "U at infinity is : " , u_inf_


#Find line through t2 and b2
l_t2_b2 = np.cross(t2_,b2_)
l_t2_b2 = make_homog(l_t2_b2)
print "l_t2_b2 : " , l_t2_b2

#Find line through t1 and b1
l_t1_b1 = np.cross(t1_,b1_)
l_t1_b1 = make_homog(l_t1_b1)
print "l_t1_b1 : " , l_t1_b1

#Find line through u and t1
l_u_t1 = np.cross(t1_,u_inf_)
l_u_t1 = make_homog(l_u_t1)
print "l_u_t1 : " , l_u_t1

# Find tilda_t1 by the intersection of l_u_t1 and l_v_b2
tilda_t1 = np.cross(l_u_t1,l_t2_b2)
tilda_t1_ = make_homog(tilda_t1)
print "~t1 is : " , tilda_t1_

# Find v by the intersection of l_t1_b1 and l_t2_b2
v = np.cross(l_t1_b1,l_t2_b2)
v_ = make_homog(v)
print "V : " , v_


l :  [  0.00000000e+00  -9.54198473e-04   1.00000000e+00]
l_b1_b2 :  [  8.43046498e-06  -5.61577748e-04   1.00000000e+00]
U at infinity is :  [ -4.88070968e+04   1.04800000e+03   1.00000000e+00]
l_t2_b2 :  [ -4.34144653e-04  -4.39640155e-06   1.00000000e+00]
l_t1_b1 :  [ -5.84259778e-03   1.59961609e-04   1.00000000e+00]
l_u_t1 :  [ -4.54450511e-05  -3.07064982e-03   1.00000000e+00]
~t1 is :  [  2.30042665e+03   2.91618077e+02   1.00000000e+00]
V :  [  1.72766771e+03   5.68515633e+04   1.00000000e+00]


In [101]:
plt.imshow(img_data)
plt.scatter(tilda_t1_[0],tilda_t1_[1])
plt.show()

/home/anand/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [109]:
# Represent points by their distance from b2

b2_dist = distance_from_b2(b2_)
tilda_t1_dist = distance_from_b2(tilda_t1_)
t2_dist = distance_from_b2(t2_)
v_dist = distance_from_b2(v_)
print "V : ", v_dist

H = [[1,v_dist[0]],[0,1]]
print "H : " , H

T2 = np.dot(H,v_dist)
print "T2 : " , T2

# Ratio of distance
d1_d2 = (tilda_t1_dist[0]*(v_dist[0] - t2_dist[0]))/(t2_dist[0]*(v_dist[0] - tilda_t1_dist[0]))
print "Ratio of distance d1/d2 = " , d1_d2

calculated_dist = 125/d1_d2

print "Length of 5-7 in cm : " , calculated_dist , "cm"

# plt.imshow(img_data)
# plt.scatter(v_dist[0],v_dist[1])
# plt.show()

V :  [  5.50393851e+04   1.00000000e+00]
H :  [[1, 55039.385114069162], [0, 1]]
T2 :  [  1.10078770e+05   1.00000000e+00]
Ratio of distance d1/d2 =  3.93798491043
Length of 5-7 in cm :  31.7421226447 cm


In [111]:
correct_distance = 32.7
print "Error : " , correct_distance - calculated_dist , "cm"

Error :  0.957877355255 cm


In [113]:
print "The error can be because of not having sub-pixel accuracy and approximation."

The error can be because of not having sub-pixel accuracy and approximation.
